In [2]:
global image_dictionary
image_dictionary = {}

In [3]:
global blure_image_dict
blur_image_dict = {}

In [47]:
import numpy as np
from numpy.fft import fft2 , ifft2
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import cv2
import numpy.fft as fp

import scipy.misc

from numpy import sqrt

from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import data , feature,restoration
from skimage import exposure
from skimage.exposure import match_histograms
from skimage.feature import canny,blob_log

from skimage.filters import sobel, threshold_otsu
from scipy.signal import butter as butterworth
from homofilt import HomomorphicFilter
from scipy import ndimage,misc

from PIL import Image, ImageFilter

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import io
from PIL import Image
from skimage.color import rgb2gray

from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim

In [5]:
def histogram_xy(img):
    # img = img.astype(int)
    if img.ndim >2:
        # print(img)
        img = np.float32(img)
        img = cv2.cvtColor(np.array(img),cv2.COLOR_RGB2GRAY)
        # print(img)
    y, bin_edges = np.histogram(img.flatten(), bins=256, range=(0, 256))
    return bin_edges[0:-1],y

In [6]:
def on_cliked_button(img,title):
    # try:
        # a = {}
        
        img = Image.open(io.BytesIO(list(img.values())[-1]['content']))
        img = np.array(img)
        # img = cv2.imread(io.BytesIO(list(img.values())[-1]['content']))
        image_dictionary.update({title:img})
        # print(image_dictionary)
        plt.imshow(img)
        # print(np.array(img))
    # except:
        # print("No File Uploaded")
    

In [7]:
interact_manual(on_cliked_button,img=widgets.FileUpload(),title=widgets.Textarea());

interactive(children=(FileUpload(value={}, description='Upload'), Textarea(value='', description='title'), But…

In [9]:
len(image_dictionary)

6

## Homomorphic

In [10]:
def homomorphic_filter(img,a=1,b=1,f=30,n=2):
    """
    filter_params: Parameters to be used on filters:
                butterworth:
                    f: Cutoff frequency 
                    n: Order of filter
    """
    homo_filter = HomomorphicFilter(a = a, b = b)
    homo_img = np.zeros_like(img)
    if img.ndim < 3:
        homo_img[:,:] = homo_filter.filter(I=img[:,:], filter_params=[f,n])
    else:
        for i in range(img.ndim):
            homo_img[:,:,i] = homo_filter.filter(I=img[:,:,i], filter_params=[f,n])
    return homo_img.astype(int)

In [11]:
def homomorphic_show(img,g_low,g_high,fc):
    title = img
    img = image_dictionary[img]
    homo = homomorphic_filter(img,g_low,g_high,fc)
    # print(type(homo))
    if img.ndim < 3:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20,6))
        ax1.imshow(homo,cmap='gray')
        ax1.set_title(f'filtered image {title}')
        
        x,y = histogram_xy(homo)
        # print(f'{x},{y}')
        ax2.bar(x,y)
        ax2.set_title(f'filtered histogram image {title}')
        ssim1 = ssim(img,homo)
    else:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20,6))
        ax1.imshow(homo)
        ax1.set_title(f'filtered image {title}')
        
        x,y = histogram_xy(homo)
        ax2.bar(x,y)
        # print(f'{x},{y}')
        ax2.set_title(f'filtered histogram image {title}')
        # plt.imshow(homomorphic_filter(img,g_low,g_high,fc))
        ssim1 = ssim(img,homo,channel_axis=2)
    
    
    snr = mse(img,homo)
    
    
    print('**********************        SNR is {:.2f}   ,   ssim is {:.2f}        **********************'.format(snr,ssim1))
    

In [13]:
list_tuple = [(k, v) for k, v in image_dictionary.items()]
widgets.interact(homomorphic_show,img=widgets.RadioButtons(
    description='img: ',
    options=list(image_dictionary.keys())
),g_low=widgets.FloatSlider(
    value=1,
    min=.01,
    max=5.0,
    step=0.01,
    description='g_low:'),g_high=widgets.FloatSlider(
    value=1,
    min=.01,
    max=5.0,
    step=0.01,
    description='g_high:'),fc=widgets.FloatSlider(
    value=30,
    min=1,
    max=100,
    step=1,
    description='fc:'))

interactive(children=(RadioButtons(description='img: ', options=('a1', 'a2', 'low_contrast', 'x3', 'x4', 'x5')…

<function __main__.homomorphic_show(img, g_low, g_high, fc)>

## Histogram equalizer

In [14]:
def histogram_equalizer(img):
    img2 = img.copy()
    for i in range(3):
        hist,bins = np.histogram(img[...,i].flatten(),256,[0,256])
        cdf = hist.cumsum()
        cdf_m = np.ma.masked_equal(cdf,0)
        cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())

        cdf = np.ma.filled(cdf_m,0).astype('uint8')
        img2[...,i] = cdf[img[...,i]]
    return img2

In [15]:
def equalize(img):
    title = img
    img = image_dictionary[img]
    img_hist_x,img_hist_y = histogram_xy(img)
    
    img_eq = histogram_equalizer(img)
    img_eq_hist_x,img_eq_hist_y = histogram_xy(img_eq)
    
    fig, axs = plt.subplots(2, 2, figsize = (20,10))
    axs[0, 0].imshow(img)
    axs[0, 0].set_title(f'Original image {title}')
    axs[0, 1].bar(img_hist_x,img_hist_y)
    axs[0, 1].set_title(f'Histogram of {title}')
    axs[1, 0].imshow(img_eq)
    axs[1, 0].set_title(f'Equalized image {title}')
    axs[1, 1].bar(img_eq_hist_x, img_eq_hist_y)
    axs[1, 1].set_title(f'Histogram of equalized image {title}')
    
    snr = mse(img,img_eq)
    if img_eq.ndim > 2:
        ssim1 = ssim(img,img_eq,channel_axis=2)
    else:
        ssim1 = ssim(img,img_eq,channel_axis=1)
    
    print('**********************        SNR is {:.2f}   ,   ssim is {:.2f}        **********************'.format(snr,ssim1))
    

In [16]:
list_tuple = [(k, v) for k, v in image_dictionary.items()]
widgets.interact(equalize,img=widgets.RadioButtons(
    description='img: ',
    options=list(image_dictionary.keys())
))

interactive(children=(RadioButtons(description='img: ', options=('a1', 'a2', 'low_contrast', 'x3', 'x4', 'x5')…

<function __main__.equalize(img)>

## Histogram matching

In [17]:
def histogram_match(img,img_match):
    title = img
    img = image_dictionary[img]
    
    # print(img.max())
    
    title_match = img_match
    img_match = image_dictionary[img_match]
    # print(img_match.max())
    
    print(img_match.shape)
    cmap = "viridis"
    if img.ndim > img_match.ndim:
        img_match = cv2.cvtColor(img_match, cv2.COLOR_BGR2RGB)
        # cmap = "gray"
    elif img.ndim < img_match.ndim:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cmap = "gray"
    # else:
    #     cmap = "viridis"
     
    img_hist_x,img_hist_y = histogram_xy(img)
    img_match_x,img_match_y = histogram_xy(img_match)
    
    matched = match_histograms(img, img_match)
    
    matched = matched.astype(int)
    
    matched_hist_x,matched_hist_y = histogram_xy(matched)
    
    fig, (ax1,ax2,ax3,ax4) = plt.subplots(1, 4, figsize = (20,5))
    ax1.imshow(img,cmap=cmap)
    ax1.set_title(f'Original image {title}')
    ax2.bar(img_hist_x,img_hist_y)
    ax2.set_title(f'Histogram of {title}') 
    ax3.imshow(img_match,cmap=cmap)
    ax3.set_title(f'Original image {title_match} for match')
    ax4.bar(img_match_x,img_match_y)
    ax4.set_title(f'OHistogram of {title_match} for match')
    fig, (ax5,ax6) = plt.subplots(1, 2, figsize = (20,6))
    ax5.imshow(matched,cmap=cmap)
    ax5.set_title('Matched image')
    ax6.bar(matched_hist_x,matched_hist_y)
    ax6.set_title('Histogram of Matched image') 
    
    snr = mse(img,matched)
    if img.ndim > 2:
        ssim1 = ssim(img,matched,channel_axis=2)
    else:
        ssim1 = ssim(img,matched,channel_axis=1)
    # ssim1 = ssim(img,matched,multichannel=True)
    
    print('**********************        SNR is {:.2f}   ,   ssim is {:.2f}        **********************'.format(snr,ssim1))
    

In [18]:
list_tuple = [(k, v) for k, v in image_dictionary.items()]
widgets.interact(histogram_match,img=widgets.RadioButtons(
    description='img: ',
    value = 'low_contrast',
    options=list(image_dictionary.keys())
),img_match=widgets.RadioButtons(
    description='match: ',
    value = 'x3',
    options=list(image_dictionary.keys())
))

interactive(children=(RadioButtons(description='img: ', index=2, options=('a1', 'a2', 'low_contrast', 'x3', 'x…

<function __main__.histogram_match(img, img_match)>

# section2


## Canny

In [19]:
def canny_filter(img,sigma,low_threshold,high_threshold):
    if low_threshold >= high_threshold:
        print('! Error : the high_threshold must grater than low_threshold',end='\r')
        return None
    else:
        print('ok',end='\r')
    # print('masoud')
    title = img
    img = image_dictionary[img]
    
    cmap = "gray"
    if img.ndim > 2:
        # img = np.float32(img)
        img = cv2.cvtColor(np.array(img),cv2.COLOR_RGB2GRAY)
    
    edges = canny(
                image=img,
                sigma=sigma,
                low_threshold=low_threshold,
                high_threshold=high_threshold)
    
    fig ,(ax1,ax2) = plt.subplots(1,2,figsize=(20,10))
    ax1.imshow(img,cmap=cmap)
    ax1.set_title(f'Original image {title}')
    ax2.imshow(edges,cmap=cmap)
    ax2.set_title(f'edges detected of {title}')
    

In [20]:
list_tuple = [(k, v) for k, v in image_dictionary.items()]

widgets.interact(canny_filter,
                img=widgets.RadioButtons(description='img: ',value = 'x4',options=list(image_dictionary.keys())),
                sigma=widgets.FloatSlider(value=1,min=.01,max=10.0,step=0.01,description='sigma:'),
                low_threshold=widgets.IntSlider(value=100,min=1,max=256,step=1,description='low_threshold:'),
                high_threshold=widgets.IntSlider(value=200,min=0,max=256,step=1,description='high_threshold:'),
                
                )

interactive(children=(RadioButtons(description='img: ', index=4, options=('a1', 'a2', 'low_contrast', 'x3', 'x…

<function __main__.canny_filter(img, sigma, low_threshold, high_threshold)>

## Marr_Hildreth

In [21]:

def laplace_of_gaussian(gray_img, sigma=1., kappa=0.75, pad=False):
    """
    Applies Laplacian of Gaussians to grayscale image.

    :param gray_img: image to apply LoG to
    :param sigma:    Gauss sigma of Gaussian applied to image, <= 0. for none
    :param kappa:    difference threshold as factor to mean of image values, <= 0 for none
    :param pad:      flag to pad output w/ zero border, keeping input image size
    """
    assert len(gray_img.shape) == 2
    img = cv2.GaussianBlur(gray_img, (0, 0), sigma) if 0. < sigma else gray_img
    img = cv2.Laplacian(img, cv2.CV_64F)
    rows, cols = img.shape[:2]
    # min/max of 3x3-neighbourhoods
    min_map = np.minimum.reduce(list(img[r:rows-2+r, c:cols-2+c]
                                     for r in range(3) for c in range(3)))
    max_map = np.maximum.reduce(list(img[r:rows-2+r, c:cols-2+c]
                                     for r in range(3) for c in range(3)))
    # bool matrix for image value positiv (w/out border pixels)
    pos_img = 0 < img[1:rows-1, 1:cols-1]
    # bool matrix for min < 0 and 0 < image pixel
    neg_min = min_map < 0
    neg_min[1 - pos_img] = 0
    # bool matrix for 0 < max and image pixel < 0
    pos_max = 0 < max_map
    pos_max[pos_img] = 0
    # sign change at pixel?
    zero_cross = neg_min + pos_max
    # values: max - min, scaled to 0--255; set to 0 for no sign change
    value_scale = 255. / max(1., img.max() - img.min())
    values = value_scale * (max_map - min_map)
    values[1 - zero_cross] = 0.
    # optional thresholding
    if 0. <= kappa:
        thresh = float(np.absolute(img).mean()) * kappa
        values[values < thresh] = 0.
    log_img = values.astype(np.uint8)
    if pad:
        log_img = np.pad(log_img, pad_width=1, mode='constant', constant_values=0)
    return log_img


In [22]:
def Marr_Hildreth(img,sigma,kappa):
    
    title = img
    img = image_dictionary[img]
    
    if img.ndim > 2:
        img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        
    
    # apply LoG
    log = laplace_of_gaussian(img,sigma=sigma,kappa=kappa)
   
    fig,(ax1,ax2) = plt.subplots(1,2,figsize = (25,15))

    ax1.imshow(img,cmap='gray')
    ax1.set_title(f'Original image {title}')
    ax2.imshow(log,cmap='gray')
    ax2.set_title(f'edges of image {title}')


In [23]:
list_tuple = [(k, v) for k, v in image_dictionary.items()]

widgets.interact(Marr_Hildreth,
                img=widgets.RadioButtons(description='img: ',value = 'x4',options=list(image_dictionary.keys())),
                sigma=widgets.FloatSlider(value=1,min=.01,max=10.0,step=0.01,description='sigma:'),
                kappa=widgets.FloatSlider(value=10,min=.5,max=100,step=.1,description='diff_threshold:'),  
                )

interactive(children=(RadioButtons(description='img: ', index=4, options=('a1', 'a2', 'low_contrast', 'x3', 'x…

<function __main__.Marr_Hildreth(img, sigma, kappa)>

# Section 3

In [24]:
def median(img,sz):
    title = img
    img = image_dictionary[img]
    
    if img.ndim > 2:
        img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    result = ndimage.median_filter(img, size=sz)
    
    img_hist_x,img_hist_y = histogram_xy(img)
    hist_x,hist_y = histogram_xy(result)
    fig,ax = plt.subplots(2,2,figsize = (25,15))

    ax[0,0].imshow(result,cmap='gray')
    ax[0,0].set_title(f'filtered image {title}')
    ax[0,1].bar(hist_x,hist_y)
    ax[0,1].set_title(f'Histogram filtered image {title}')
    ax[1,0].imshow(img,cmap='gray')
    ax[1,0].set_title(f'Original image {title}')
    ax[1,1].bar(img_hist_x,img_hist_y)
    ax[1,1].set_title(f'Histogram Original image {title}')
    # plt.imshow(result,cmap='gray')

In [25]:
list_tuple = [(k, v) for k, v in image_dictionary.items()]

widgets.interact(median,
                img=widgets.RadioButtons(description='img: ',value = 'x4',options=list(image_dictionary.keys())),
                sz=widgets.IntSlider(value=3,min=1,max=15,step=2,description='size:'), 
                )

interactive(children=(RadioButtons(description='img: ', index=4, options=('a1', 'a2', 'low_contrast', 'x3', 'x…

<function __main__.median(img, sz)>

# section 4

## image blurring

In [26]:
def motion_blur(img,kind,sz):
    kernel_size = sz
  
    # Create the kernel.
    kernel_v = np.zeros((kernel_size, kernel_size))
    kernel_h = np.copy(kernel_v)

    # Fill the middle row with ones.
    kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)
    kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size)

    # Normalize.
    kernel_v /= kernel_size
    kernel_h /= kernel_size
    
    if kind == 'v':
        vertical_mb = cv2.filter2D(img, -1, kernel_v)
        return vertical_mb
    elif kind == 'h':
        horizonal_mb = cv2.filter2D(img, -1, kernel_h)
        return horizonal_mb
    else:
        print("the kind of kernel is wrong")

In [27]:
def do_bluring(img,blur,sz):
    title = img
    img = image_dictionary[img]

    blured_title = f'blured_{title}'
    
    if blur == 'motion_horizontal':
        blured = motion_blur(img,'h',sz)
    elif blur == 'motion_vertical':
        blured = motion_blur(img,'v',sz)
    elif blur == 'out_of_focus':
        blured = motion_blur(img,'h',sz)
        blured = motion_blur(blured,'v',sz)
    else:
        print('error')
        # return None
       
    blured_hist_x,blured_hist_y = histogram_xy(blured)
    img_his_x,img_his_y = histogram_xy(img)
    # blured = motion_blur(img,'h',sz)
    
    if blured_title in blur_image_dict:
        blur_image_dict[blured_title] = blured
    else:
        blur_image_dict.update({blured_title:blured})
    
    # print(img)
    
    cmap = "viridis"
    if img.ndim < 3 :
        cmap = "gray"
    fig,ax = plt.subplots(2,2,figsize = (25,15))
    ax[0,0].imshow(img,cmap=cmap)
    ax[0,0].set_title(f'Original image {title}')
    ax[0,1].imshow(blured,cmap=cmap)
    ax[0,1].set_title(f'blured image {title}')
    ax[1,0].bar(img_his_x,img_his_y)
    ax[1,0].set_title(f'Histogram of Original image {title}')
    ax[1,1].bar(blured_hist_x,blured_hist_y)
    ax[1,1].set_title(f'Histogram of blured image {title}')
    

In [28]:
list_tuple = [(k, v) for k, v in image_dictionary.items()]

widgets.interact(do_bluring,
                img=widgets.RadioButtons(description='img: ',value = 'x4',options=list(image_dictionary.keys())),
                blur=widgets.RadioButtons(description='blur: ',options=['motion_horizontal','motion_vertical','out_of_focus']),
                sz=widgets.IntSlider(value=3,min=1,max=200,step=2,description='size:')
                )

interactive(children=(RadioButtons(description='img: ', index=4, options=('a1', 'a2', 'low_contrast', 'x3', 'x…

<function __main__.do_bluring(img, blur, sz)>

## wiener filter

In [111]:
from scipy.signal import gaussian, convolve2d

In [112]:
def wiener_filter(img, kernel, K):
    kernel /= np.sum(kernel)
    dummy = np.copy(img)
    dummy = fft2(dummy)
    kernel = fft2(kernel, s = img.shape)
    kernel = np.conj(kernel) / (np.abs(kernel) ** 2 + K)
    dummy = dummy * kernel
    dummy = np.abs(ifft2(dummy))
    return dummy

In [113]:
def gaussian_kernel(kernel_size = 3):
    h = gaussian(kernel_size, kernel_size / 3).reshape(kernel_size, 1)
    h = np.dot(h, h.transpose())
    h /= np.sum(h)
    return h

In [114]:
def wiener(img,sz):
    title = img
    img = blur_image_dict[title]
    img = img.astype(float)
    
    org_img = title.replace('blured_', '')
    org_img = image_dictionary[org_img]
    
    org_img = org_img.astype(float)
    
    
    
    K = 10
    kernel = gaussian_kernel(sz)
    restored = wiener_filter(img, kernel, K = K)

    
    
#     sigma_est = np.mean(estimate_sigma(img, channel_axis=2))
#     restored = denoise_nl_means(img, h=sz * sigma_est, fast_mode=False,
#                                 patch_size=5, patch_distance=3, multichannel=False)
    
    
    
    # base_image = image_dictionary['x3']
    # param = 50
#     psf = np.ones((sz, sz)) / 25
#     restored,_ = restoration.unsupervised_wiener(img,psf)
    
    # restored = wiener_filter(img,base_image)
    
#     from scipy.signal.signaltools import wiener
#     restored = wiener(img, (sz, sz))
    
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(20,10))
    ax1.imshow(restored,cmap='gray')
    ax2.imshow(img,cmap='gray')   
    # print(img.replace('blured_', ''))
    # img = image_dictionary[img]
    
    
    pass

In [115]:
widgets.interact(wiener,
                img=widgets.RadioButtons(description='img: ',options=list(blur_image_dict.keys())),
                sz=widgets.IntSlider(value=3,min=1,max=100,step=2,description='size:')
                )

interactive(children=(RadioButtons(description='img: ', options=('blured_x4',), value='blured_x4'), IntSlider(…

<function __main__.wiener(img, sz)>

In [107]:
from astroML.filters import savitzky_golay, wiener_filter

In [88]:
import SimpleITK as sitk

In [ ]:
tkﬁlter = sitk.WienerDeconvolutionImageFilter()
tkﬁlter.SetNoiseVariance(0)
tkﬁlter.SetNormalize(True)
im_res_WN = sitk.GetArrayFromImage(tkﬁlter.Execute
(sitk.GetImageFromArray(im_blur),
sitk.GetImageFromArray(psf)))